## Лаба 2. Content-based рекомендательная система образовательных курсов – Spark Dataframes

#### Дедлайн

Понедельник,  08 марта, 23:59:59.

#### Дедлайн Github

Четверг, 11 марта, 23:59:59.

#### Задача

По имеющимся данным портала eclass.cc построить content-based рекомендации по образовательным курсам. Запрещено использовать библиотеки pandas, sklearn и аналогичные.

#### Описание данных

Имеются следующие данные на вход:

* набор данных о всех курсах. Датасет можно взять с HDFS по адресу: `/labs/slaba02/DO_record_per_line.json`
* id курсов, для которых надо дать рекомендации (указаны в [Личном кабинете](https://lk-spark.newprolab.com/lab/slaba02)).

Данные выглядят следующим образом:

```json
{"lang": "en", 
"name": "Accounting Cycle: The Foundation of Business Measurement and Reporting", 
"cat": "3/business_management|6/economics_finance", 
"provider": "Canvas Network", 
"id": 4, 
"desc": "This course introduces the basic financial statements used by most businesses, as well as the essential tools used to prepare them. This course will serve as a resource to help business students succeed in their upcoming university-level accounting classes, and as a refresher for upper division accounting students who are struggling to recall elementary concepts essential to more advanced accounting topics. Business owners will also benefit from this class by gaining essential skills necessary to organize and manage information pertinent to operating their business. At the conclusion of the class, students will understand the balance sheet, income statement, and cash flow statement. They will be able to differentiate between cash basis and accrual basis techniques, and know when each is appropriate. They\u2019ll also understand the accounting equation, how to journalize and post transactions, how to adjust and close accounts, and how to prepare key financial reports. All material for this class is written and delivered by the professor, and can be previewed here. Students must have access to a spreadsheet program to participate."}
```

#### Результат

Для каждого id курса из личного кабинета необходимо дать топ-10 наиболее похожих на него курсов. Рекомендованные курсы должны быть того же языка, что и курс, для которого строится рекомендация.

Выходной формат — json — должен иметь следующую структуру:

```
{
  "123": [
    5372,
    16663,
    23114,
    13079,
    13084,
    ...
  ],
  "456": [
    ...
  ],
  "789": [
    ...
  ],
  "123456": [
    ...
  ],
  "456789": [
    ...
  ],
  "987654": [
    ...
  ]
}
```

Ключи json — это id курсов, для которых строится рекомендация. Для каждого такого ключа в качестве значения задается массив рекомендованных курсов, состоящий из их id, отсортированных по убыванию метрики. При равенстве значений метрики курсы сортируются лексикографически по названию.

Также возможна очень редкая ситуация (в основном с русскоязычными курсами), когда в рекомендацию попадут два дубликата одного курса, но с разными id. Таких дубликатов очень мало относительно числа курсов, но все равно рекомендуется сортировать в следующей последовательности: по метрике (убывание) => по названию (лексикографически по возрастанию) => по возрастанию id.

Также вы можете найти так называемый submission-файл по следующему пути на мастер-ноде: `/share/submission-files/slaba02/lab02.json`. Он обладает правильной структурой и форматом, но неправильными значениями. Идея в том, что он проходит необходимые требования чекера именно по структуре, и вам не придется пытаться понять, в чем дело, почему чекер не принимает ваш файл.

#### Советы

Для подбора рекомендаций следует использовать меру TFIDF, а в качестве метрики для ранжирования — косинус угла между TFIDF-векторами для разных курсов.

Что такое TFIDF? TF — это term frequency: по сути, сколько раз слово встречается в этом документе. Если мы сделаем такой word count по каждому документу, то получим вектор, который как-то характеризует этот документ. 

```
мама - 2
мыла - 1
раму - 1
лапу - 1
роза - 2
упала - 3
```

Если мы сравним вектора, рассчитав дистанцую между ними, то получим, насколько похожи эти тексты. Назовем этот подход наивным.

Этот подход наивен, потому что мы как бы присваиваем одинаковый вес каждому слову, которое у нас есть в тексте. А что если мы попробуем как-то повысить значимость тех слов, которые часто встречаются только в этом тексте? Для этого мы посчитаем DF – document frequency: по сути, число документов, в которых есть вхождение этого слова. Мы хотим "штрафовать" слово за частое появление в документах, поэтому делаем инверсию этой величины – буква I в TFIDF. Теперь для каждого слова мы будем считать TF и делить на IDF. Так мы получим другой вектор для нашего документа. Он может быть более правильным для наших задач.

TFIDF нужно считать для описаний курсов (desc). При извлечении слов из описания словом считаем то, что состоит из латинских или кириллических букв или цифр, знаки препинания и прочие символы не учитываются.

Для поиска слов можно использовать такой код на Python (может быть проблема с распознаванием юникода). 

```
regex = re.compile(u'[\w\d]{2,}', re.U)
regex.findall(string.lower())
```

Сам TFIDF реализован в Spark, писать с нуля вычисления не требуется. При вычислении TF с помощью `HashingTF` использовалось число фичей: 10000. То есть: `tf = HashingTF(10000)`.

#### Проверка

Проверка осуществляется по результатам рекомендаций текущей рекомендательной системы на eclass.cc. Для прохождения лабораторной для каждого курса, для которого строится рекомендация, должно быть пересечение рекомендованных курсов с результатами текущей системы — **не менее 20%**.

Файл необходимо положить в свою домашнюю директорию под названием: `lab02.json`. Проверка осуществляется со страницы лабы в личном кабинете. В чекере в качестве значений для курсов указаны id и доля пересечения конкретно для каждого из курсов.

Обязательное условие зачета лабораторной работы – это выкладка после дедлайна лабы своего решения в репозиторий через pull-request. Как это сделать, можно прочитать [здесь](/git.md). Если будут вопросы – спрашивайте в Slack.

# Lab02 Решение

In [2]:
import os
import sys

os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [4]:
data = spark.read.json('/labs/slaba02/DO_record_per_line.json')

In [5]:
data.count()

28153

In [6]:
data.rdd.getNumPartitions()

3

In [7]:
data.printSchema()

root
 |-- cat: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- id: long (nullable = true)
 |-- lang: string (nullable = true)
 |-- name: string (nullable = true)
 |-- provider: string (nullable = true)



In [8]:
# Курсы, по которым нужно выдать решение
given_courses =  [
 [23126, u'en', u'Compass - powerful SASS library that makes your life easier'], 
 [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], 
 [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], 
 [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], 
 [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], 
 [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]

In [9]:
given_courses

[[23126, 'en', 'Compass - powerful SASS library that makes your life easier'],
 [21617, 'en', 'Preparing for the AP* Computer Science A Exam — Part 2'],
 [16627, 'es', 'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'],
 [11556,
  'es',
  'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'],
 [16704, 'ru', 'Программирование на Lazarus'],
 [13702, 'ru', 'Математическая экономика']]

In [10]:
courses_langs = [(course[0], course[1]) for course in given_courses]

## HashingTF + TFIDF + dot product + l2_norm

In [11]:
from pyspark.ml.feature import HashingTF, IDF

In [12]:
from pyspark.sql.types import ArrayType, StringType, FloatType

In [13]:
import pyspark.sql.functions as f
from pyspark.sql.functions import pandas_udf
import re

def clear_string(series):
    regex = re.compile(u'[\w\d]{2,}', re.U)
    words = series.str.findall(regex)
    return words

tokenizer_udf = pandas_udf(clear_string, ArrayType(StringType()))

In [14]:
tokenized_data = data.withColumn("words", tokenizer_udf("desc"))

In [15]:
hashingTF = HashingTF(inputCol="words", outputCol="TFFeatures", numFeatures=10000, binary=False)

hashed_data = hashingTF.transform(tokenized_data)

In [16]:
idf = IDF(inputCol="TFFeatures", outputCol="features")
idfModel = idf.fit(hashed_data)
idfed_data = idfModel.transform(hashed_data)

In [17]:
@pandas_udf(ArrayType(FloatType()))
def vectorToArray(row):
    return row.tolist()

In [18]:
%%time

# Нормализация векторов L2, после этого для cosine_similarity будет достаточно 
# делать dot product нормализованных векторов
from pyspark.ml.feature import Normalizer
t = Normalizer(inputCol='features', outputCol='norm_features', p=2.0)

normalized_data = t.transform(idfed_data)
# Для каждого курса посчитаем косинусное расстояние до всех остальных
# выберем 10 самых похожих

target_course_reqs = {}

for course_id, lang in courses_langs:
    course_vec = normalized_data.filter(normalized_data.id == int(course_id))\
             .collect()[0]['features'].toArray()
    
    cos_sim = f.udf(lambda x: float(x.dot(course_vec)), FloatType())
    
    recs = normalized_data.where((normalized_data.id != int(course_id)) & (normalized_data.lang == lang))\
               .withColumn('cosine_sim', cos_sim(normalized_data['features']))\
               .orderBy(f.desc('cosine_sim'), f.asc('name'), f.asc('id'))\
               .head(10)
                           
    list_out = [rec['id'] for rec in recs]
    target_course_reqs.update({str(course_id): list_out})

CPU times: user 520 ms, sys: 178 ms, total: 698 ms
Wall time: 1min 7s


In [19]:
target_course_reqs

{'23126': [13665, 11063, 23257, 6938, 3919, 3819, 11616, 25782, 10950, 12465],
 '21617': [16971, 17221, 6776, 380, 7597, 19848, 22297, 116, 8110, 21081],
 '16627': [26336,
  26670,
  7944,
  17839,
  21053,
  10749,
  23303,
  18979,
  23118,
  11575],
 '11556': [26336, 26670, 7944, 16929, 21053, 17839, 10749, 18979, 3660, 8098],
 '16704': [1247, 5221, 1234, 20319, 8203, 1365, 8217, 7604, 25830, 22553],
 '13702': [28074, 864, 28075, 25830, 22553, 19648, 5221, 7611, 7607, 8313]}

In [20]:
import json
with open('../lab02.json', 'w') as fout:
    fout.write(json.dumps(target_course_reqs))

In [21]:
spark.stop()